# Импорт библиотек

In [2]:
import torch #as keras or tensorflow
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms #reform data for cnn
import torchvision.datasets
from bokeh.plotting import figure #as matplotlib to html graphic
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np #for math and nD_arrays

# Определение гиперпараметров

In [3]:
num_epochs = 6 
num_classes = 10 #out layer
batch_size = 100 
learning_rate = 0.001 

DATA_PATH = 'data'
MODEL_STORE_PATH = 'models'

# Преобразование данных

In [4]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Загрузка датасета

In [5]:
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



# Импорт датасета

In [6]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Двуслойная свёрточная нейронная сеть

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x): 
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
model = ConvNet()

# Функция ошибки и оптимизатор

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Тренеровка нейросети

In [10]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/6], Step [100/600], Loss: 0.2605, Accuracy: 90.00%
Epoch [1/6], Step [200/600], Loss: 0.2433, Accuracy: 93.00%
Epoch [1/6], Step [300/600], Loss: 0.1482, Accuracy: 96.00%
Epoch [1/6], Step [400/600], Loss: 0.1729, Accuracy: 95.00%
Epoch [1/6], Step [500/600], Loss: 0.0609, Accuracy: 98.00%
Epoch [1/6], Step [600/600], Loss: 0.2238, Accuracy: 95.00%
Epoch [2/6], Step [100/600], Loss: 0.2078, Accuracy: 92.00%
Epoch [2/6], Step [200/600], Loss: 0.0304, Accuracy: 100.00%
Epoch [2/6], Step [300/600], Loss: 0.0879, Accuracy: 97.00%
Epoch [2/6], Step [400/600], Loss: 0.0637, Accuracy: 98.00%
Epoch [2/6], Step [500/600], Loss: 0.0616, Accuracy: 97.00%
Epoch [2/6], Step [600/600], Loss: 0.0834, Accuracy: 97.00%
Epoch [3/6], Step [100/600], Loss: 0.1158, Accuracy: 97.00%
Epoch [3/6], Step [200/600], Loss: 0.0558, Accuracy: 98.00%
Epoch [3/6], Step [300/600], Loss: 0.0860, Accuracy: 97.00%
Epoch [3/6], Step [400/600], Loss: 0.1749, Accuracy: 96.00%
Epoch [3/6], Step [500/600], Loss: 0.08

# Тест на новых данных

In [11]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 99.08 %


# Сохранение модели и весов

In [12]:
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

# Построение графика точности 

In [13]:
p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(acc_list) * 100, y_range_name='Accuracy', color='red')
show(p)

Found ffmpeg: /opt/yandex/browser/libffmpeg.so
	avcodec: 3876708
	avformat: 3874148
	avutil: 3743332
Ffmpeg version is OK! Let's use it.
[9505:9505:1230/134358.651720:ERROR:brand_package_util.cc(112)] Failed to verify partner_config signature, config will be ignored
[9505:9505:1230/134358.681558:ERROR:variations_seed_processor.cc(253)] Trial from abt study=Spaces already created
[9505:9505:1230/134358.883224:ERROR:isolated_origin_util.cc(74)] Ignoring port number in isolated origin: chrome://custo
[9505:9505:1230/134358.883260:ERROR:child_process_security_policy_impl.cc(2010)] Invalid isolated origin: 
libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)


Окно или вкладка откроются в текущем сеансе браузера.
